In [1]:
print("STARTO")
import time
import os
start_time = time.time()

using_gpu = False
if not using_gpu:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
    os.environ["CUDA_VISIBLE_DEVICES"] = "5"

STARTO


In [2]:
# raise ValueError("test")

In [3]:
from lib.detector.SSD.ssd import Detector
ssd = Detector('./model_data/frozen_inference_graph.pb')

In [4]:
import numpy as np
import json
with open('./train_val_config/ua_voc_4.json', 'r') as fp:
    data = json.load(fp)
    train_imgs = data['train']
    valid_imgs = data['val']

In [5]:
import cv2
# "boxes":[], "scores":[]
labels = ['bicycle','bus','car','motorbike']
tests = valid_imgs
gt_boxes = {label:{test['filename']:[] for test in tests} for label in labels}
pred_boxes = {label:{test['filename']:{} for test in tests} for label in labels}
from pprint import pprint

for _dat in valid_imgs:
    key = _dat['filename']
    for obj in _dat['object']:
        gt_boxes[obj['name']][key].append([obj['xmin'], obj['ymin'], obj['xmax'], obj['ymax']])

    _img = cv2.imread(key)
    rgb_img = cv2.cvtColor(_img, cv2.COLOR_BGR2RGB)
#     obj['ymin'], obj['xmin'], obj['ymax'], obj['xmax']
    results = ssd.get_localization(rgb_img)
    for cls_name in results:
        pred_boxes[cls_name][key]['boxes'] = []
        for box in results[cls_name]['boxes']:
            pred_boxes[cls_name][key]['boxes'].append([box[1], box[0], box[3], box[2]])
        pred_boxes[cls_name][key]['scores'] = results[cls_name]['scores']

[166.66351631 145.0073719  271.14638686 391.78723097] , confidence:  0.89519346 ratio: 0.4233677629396322
[177.57211626 415.17308354 244.39018965 499.59525466] , confidence:  0.7236584 ratio: 0.7913816796231951
[110.55674404 252.99346447 201.78363472 496.32233381] , confidence:  0.88233644 ratio: 0.37489650103805705
[ 43.77711564   4.05417383 370.45841664 500.        ] , confidence:  0.73725367 ratio: 0.6586903182863175
no detection!
[ 17.62552187  13.81963491 315.85992873 481.94050789] , confidence:  0.9521074 ratio: 0.6370748525167116
[183.64991248 152.04742551 367.22360551 451.67019963] , confidence:  0.9517817 ratio: 0.6126622615754771
[167.53197089 209.83405411 213.396281   270.32154799] , confidence:  0.72035944 ratio: 0.7581191745252778
[138.72488961 103.86119783 215.89252353 217.15937555] , confidence:  0.6054801 ratio: 0.6810420525180596
[152.69387886 122.52346426 218.84961426 264.55533504] , confidence:  0.30554646 ratio: 0.4657481279210805
[168.55943576 280.98297119 182.4486

In [6]:
# gt_boxes = {label:{test['filename']:[] for test in tests} for label in labels}
# pred_boxes = {label:{test['filename']:{} for test in tests} for label in labels}
import json

def myconverter(o):
    if isinstance(o, np.float32):
        return float(o)
    
with open('gt_boxes_ssd.json', 'w') as outfile:
    json.dump(gt_boxes, outfile, default=myconverter)
with open('pred_boxes_ssd.json', 'w') as outfile:
    json.dump(pred_boxes, outfile, default=myconverter)

In [7]:
!mv gt_boxes_ssd.json analysis/gt_boxes_ssd.json
!mv pred_boxes_ssd.json analysis/pred_boxes_ssd.json